In [ ]:
# API를 코드 밖에서 관리하게 함
!pip install python-dotenv

In [ ]:
# XML을 dict로 바꿔줌
!pip install xmltodict

In [ ]:
# nbstripout 설치
!pip install nbstripout

# 특허·실용 공개·등록공보
- getAdvancedSearch

In [ ]:
import os
import requests
import xmltodict
import pandas as pd
from dotenv import load_dotenv

# ===============================
# 1. 환경변수 로드
# ===============================
load_dotenv()
API_KEY = os.getenv("KIPRIS_API_KEY")

if not API_KEY:
    raise RuntimeError("KIPRIS_API_KEY가 설정되어 있지 않습니다.")

# ===============================
# 2. Endpoint
# ===============================
BASE_URL = "http://plus.kipris.or.kr/kipo-api/kipi/patUtiModInfoSearchSevice/getAdvancedSearch"

# ===============================
# 3. 검색 조건
# ===============================
COMPANY_NAME = "터보파워텍"

params = {
    "ServiceKey": API_KEY,
    "applicant": COMPANY_NAME,
    "patent": "true",
    "utility": "true",
    "numOfRows": 500,
    "pageNo": 1
}

# ===============================
# 4. API 요청
# ===============================
response = requests.get(BASE_URL, params=params, timeout=10)

# HTTP 자체 실패
if response.status_code != 200:
    raise RuntimeError(
        f"HTTP 오류 발생: {response.status_code}\n{response.text}"
    )

# ===============================
# 5. XML 파싱
# ===============================
try:
    data = xmltodict.parse(response.text)
except Exception:
    raise RuntimeError(
        "XML 파싱 실패. 응답이 XML이 아닙니다.\n"
        f"응답 원문:\n{response.text}"
    )

# ===============================
# 6. API 헤더 에러 검사 (🔥 핵심)
# ===============================
header = data.get("response", {}).get("header", {})

result_code = header.get("resultCode")
result_msg = header.get("resultMsg")

if result_code and result_code != "00":
    raise RuntimeError(
        f"KIPRIS API 오류\n"
        f"resultCode: {result_code}\n"
        f"resultMsg : {result_msg}"
    )

# ===============================
# 7. body / items 검사
# ===============================
body = data.get("response", {}).get("body")

if not body:
    raise RuntimeError(
        "API body가 없습니다. 검색 조건이 인식되지 않았습니다."
    )

items = body.get("items", {}).get("item")

if not items:
    raise RuntimeError(
        f"검색 결과 없음.\n"
        f"출원인명 '{COMPANY_NAME}'이(가) "
        f"현재 파라미터로는 인식되지 않았습니다.\n\n"
        f"⚠️ applicant 파라미터가 무시되었을 가능성이 큽니다."
    )

# ===============================
# 8. DataFrame 변환 및 특정 컬럼 필터링
# ===============================
if isinstance(items, dict):
    items = [items]

# 전체 데이터를 먼저 DataFrame으로 변환
full_df = pd.DataFrame(items)

# 9개 컬럼 목록 (API 응답 필드명 기준)
target_columns = [
    "ipcNumber",          # IPC코드
    "applicationDate",    # 출원일자
    "astrtCont",          # 초록
    "applicationNumber",  # 출원번호
    "indexNo",            # 일련번호
    "registerStatus",     # 등록상태
    "inventionTitle",     # 발명의명칭
    "applicantName",      # 출원인
    "totalCount"          # 전체건수
]

# DataFrame에서 존재하는 컬럼만 선택 (방어적 선택)
# 만약 API 응답에 특정 컬럼이 없을 경우를 대비해 처리합니다.
available_columns = [col for col in target_columns if col in full_df.columns]
df = full_df[available_columns].copy()

# 출력 확인
print(f"필터링된 컬럼 수: {len(df.columns)} / 9")
df.head(5)

In [ ]:
print("실제 API 응답 컬럼 리스트:", full_df.columns.tolist())

# 특허·실용 피인용문헌
- `StandardCitationApplicationNumber`, `ApplicationNumber`

In [ ]:
import os
import requests
import xmltodict
import pandas as pd
from dotenv import load_dotenv

# ===============================
# 1. 환경변수 로드
# ===============================
load_dotenv()
API_KEY = os.getenv("KIPRIS_API_KEY")

if not API_KEY:
    raise RuntimeError("KIPRIS_API_KEY가 설정되어 있지 않습니다.")

# ===============================
# 2. Endpoint (입력값 샘플에 맞춤)
# ===============================
BASE_URL = "http://plus.kipris.or.kr/openapi/rest/CitingService/citingInfo"

# ===============================
# 3. 검색 조건 (숫자 대신 문자열로 명시)
# ===============================
APP_NUM = "1020220111724" 

params = {
    "accessKey": API_KEY,
    "standardCitationApplicationNumber": APP_NUM
}


# ===============================
# 4. API 요청
# ===============================
response = requests.get(BASE_URL, params=params, timeout=10)

if response.status_code != 200:
    raise RuntimeError(f"HTTP 오류 발생: {response.status_code}")

# ===============================
# 5. XML 파싱
# ===============================
try:
    # xmltodict는 XML의 루트 태그를 포함합니다. 
    # KIPRIS 응답의 루트는 보통 'response'입니다.
    raw_data = xmltodict.parse(response.text)
except Exception:
    raise RuntimeError(f"XML 파싱 실패. 응답 원문:\n{response.text}")

# ===============================
# 6. API 헤더 에러 검사
# ===============================
# KIPRIS Plus API는 보통 'response' -> 'header' 구조를 가집니다.
res_root = raw_data.get("response", {})
header = res_root.get("header", {})

result_code = header.get("resultCode")
result_msg = header.get("resultMsg")

# 성공 코드가 "00" 또는 "0"일 수 있으나 보통 "00"입니다.
if result_code and result_code not in ["00", "0"]:
    raise RuntimeError(f"KIPRIS API 오류: {result_code} / {result_msg}")

# ===============================
# 7. body / items 검사 (KIPRIS citingInfo 전용으로 수정)
# ===============================
body = res_root.get("body", {})
items_wrapper = body.get("items") if body else None

items = None
if items_wrapper:
    # [수정 포인트] 'item'이 아니라 'citingInfo' 태그 가져옴 (전에는 계속 결과가 나왔는데도 인용 결과가 없다고 뜸)
    items = items_wrapper.get("citingInfo")

# ===============================
# 8. DataFrame 변환 및 필터링
# ===============================
if items is None:
    print(f"⚠️ API는 성공했으나, 해당 번호({APP_NUM})의 '인용 정보'가 비어있습니다.")
    df = pd.DataFrame()
else:
    # 딕셔너리(1개)인 경우 리스트로 감싸기
    if isinstance(items, dict):
        items = [items]
    
    full_df = pd.DataFrame(items)
    
    # [중요] KIPRIS 필드명은 실제 응답에서 소문자로 올 때가 많습니다.
    # 대소문자 무관하게 매칭하기 위한 로직
    target_map = {
        "standardcitationapplicationnumber": "StandardCitationApplicationNumber",   # 선행기술 출원번호
        "applicationnumber": "ApplicationNumber"    # 후행기술 출원번호
    }
    
    # 실제 컬럼명을 소문자로 변환하여 비교
    existing_cols = {col.lower(): col for col in full_df.columns}
    
    final_cols = []
    for lower_target, original_target in target_map.items():
        if lower_target in existing_cols:
            final_cols.append(existing_cols[lower_target])
            
    if final_cols:
        df = full_df[final_cols].copy()
    else:
        # 매칭되는 컬럼이 없으면 전체라도 보여줌
        df = full_df

# 최종 결과 출력
if not df.empty:
    print(f"✅ 조회 성공! 데이터 건수: {len(df)}")
    print(df.head())
else:
    # 만약 여기서도 데이터가 없다면, 응답 원문을 출력해 구조를 확인해야 합니다.
    print("❌ 표시할 데이터가 없습니다. 응답 원문을 확인합니다:")
    print(response.text) # 원문 앞부분 출력